In [ ]:
#!/usr/bin/env python3
"""
This script creates an Obsidian PARA file structure using a tag‑based approach.
It reads CSV files and markdown pages from the 'Database Storage' folder—which contains:
  • Areas‑Resources CSV and folder
  • Notes CSV and folder
  • Projects CSV and folder
  • Tasks CSV and folder

Key updates:
  - Tasks are now inline items within project pages (and unassigned tasks are collected in a dedicated page).
  - Archive is handled solely via a "#archive" tag.
  - All tags start with "#", and tags are built only from parent area or project names.
  - YAML frontmatter for notes and projects includes a dedicated 'parent_area' property and, for notes, a 'project' property.
  - Area pages are replaced with a Map of Content (MOC) generated dynamically via Dataview queries in callouts.
  - Project pages always have inline sections "## Tasks" (with "### Incomplete Tasks" and "### Completed Tasks") and "## Notes".
  - Original files are not modified; new files are written into a flat "PARA" folder in the destination.
  
Usage:
    python migrate_para.py --source_folder <Database Storage folder path> --dest_folder <destination folder>

Requires:
    pip install python-frontmatter
"""

import os
import csv
import argparse
import shutil
import re
import frontmatter

# --- Helper Functions ---

def slugify(text):
    """Convert text to a slug: lowercase, replace spaces with underscores, remove non-alphanumeric characters."""
    text = text.lower().strip()
    text = re.sub(r'\s+', '_', text)
    text = re.sub(r'[^\w\-]', '', text)
    return text

def extract_clean_name(raw):
    """
    Extract the clean name from a raw CSV cell value.
    - If in Notion-style link format ([[...|Displayed Name]]), return the displayed name.
    - Otherwise, if the value contains " (", return the text before the parenthesis.
    - Else, return the trimmed string.
    """
    if not isinstance(raw, str):
        return ""
    raw = raw.strip()
    if raw.startswith('[[') and raw.endswith(']]'):
        inner = raw[2:-2]
        if '|' in inner:
            return inner.split('|')[-1].strip()
        return inner.strip()
    if ' (' in raw:
        return raw.split(' (')[0].strip()
    return raw

def parse_itemlist(cell_value, marker="Notes%"):
    """
    Parse a cell that may contain multiple items.
    Uses a regex to capture text before a parenthesized segment containing the marker.
    Returns a list of cleaned item names.
    """
    if not isinstance(cell_value, str):
        return []
    # Find all segments that end with a parenthesized link containing the marker.
    pattern = rf'(.*?)(?:\s*\({marker}.*?\))'
    matches = re.findall(pattern, cell_value)
    if matches:
        return [m.strip(" ,") for m in matches if m.strip()]
    else:
        # If no matches, return the entire cleaned value as a single item.
        return [cell_value.strip()]

def parse_csv_file(csv_path):
    """
    Parse a CSV file and return a list of dictionaries (one per row).
    Every string value is cleaned using extract_clean_name.
    """
    rows = []
    with open(csv_path, mode='r', encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            cleaned = { key: extract_clean_name(val) if isinstance(val, str) else val
                        for key, val in row.items() if val }
            rows.append(cleaned)
    return rows

def merge_relationship_rows(rows):
    """
    Merge a list of relationship rows for the same entity.
    For single-value fields ('Area/Resource', 'Type'), take the first value.
    For multi-value fields ('Resources', 'Notes', 'Projects'), use parse_itemlist on each and merge uniquely.
    For 'Root Area', filter out values that start with "http".
    For 'Archive', return 'Yes' if any row indicates truthy.
    """
    merged = {}
    for row in rows:
        for key, val in row.items():
            merged.setdefault(key, []).append(val)
    for field in ['Area/Resource', 'Type']:
        if field in merged:
            merged[field] = merged[field][0]
    for field, marker in [('Resources', "Areas%"), ('Notes', "Notes%"), ('Projects', "Projects%")]:
        if field in merged:
            all_items = []
            for cell in merged[field]:
                items = parse_itemlist(cell, marker=marker)
                all_items.extend(items)
            merged[field] = list(dict.fromkeys(all_items))
    if 'Root Area' in merged:
        filtered = []
        for cell in merged['Root Area']:
            clean = extract_clean_name(cell)
            if not clean.lower().startswith("http"):
                filtered.append(clean)
        merged['Root Area'] = list(dict.fromkeys(filtered))
    if 'Archive' in merged:
        merged['Archive'] = 'Yes' if any(x.lower() in ['yes', 'true', 'x'] for x in merged['Archive']) else 'No'
    return merged

def find_markdown_pages(source_folder, category):
    """
    Return a dictionary mapping clean page names (lowercase) to file paths for the given category.
    Category must be one of 'areas', 'notes', or 'projects'.
    For 'projects', only return files whose immediate parent folder is "Projects" (case-insensitive).
    """
    folder_map = {
        'areas': 'Areas-Resources',
        'notes': 'Notes',
        'projects': 'Projects'
    }
    pages = {}
    target_folder = os.path.join(source_folder, folder_map[category])
    if not os.path.isdir(target_folder):
        print(f"Warning: Folder {folder_map[category]} not found in {source_folder}")
        return pages
    for root, _, files in os.walk(target_folder):
        for file in files:
            if file.endswith(".md"):
                if category == 'projects':
                    if os.path.basename(os.path.normpath(root)).lower() != 'projects':
                        continue
                clean_name = extract_clean_name(os.path.splitext(file)[0]).lower()
                pages.setdefault(clean_name, []).append(os.path.join(root, file))
    return {k: v[0] for k, v in pages.items()}

def map_relationships(rel_data, key_field):
    """
    Map relationship data from CSV rows into a dictionary keyed by the clean page name from key_field.
    Multiple rows for the same entity are merged.
    """
    mapping = {}
    for row in rel_data:
        if key_field in row:
            clean = extract_clean_name(row[key_field]).lower()
            mapping.setdefault(clean, []).append(row)
    merged_mapping = { k: merge_relationship_rows(v) for k, v in mapping.items() }
    return merged_mapping

# --- Content Generation Functions ---

def generate_project_tasks_section(tasks_list):
    """
    Generate an inline tasks section for a project.
    Always outputs the "## Tasks" header with subsections for "### Incomplete Tasks" and "### Completed Tasks".
    Each task is output as a markdown list item with a checkbox and "#task" tag.
    Recurring tasks get the tag "#tasks/recurring".
    """
    output = "## Tasks\n\n"
    incomplete = []
    complete = []
    for task in tasks_list:
        line = "- "
        line += "[x] " if task['status'] else "[ ] "
        line += f"#task {task['task']}"
        if task.get('due'):
            line += f" (📅 {task['due']})"
        if task.get('recurring'):
            line += " (🔁 every week)"
        if task['status']:
            complete.append(line)
        else:
            incomplete.append(line)
    output += "### Incomplete Tasks\n"
    output += "\n".join(incomplete) if incomplete else "No tasks assigned.\n"
    output += "\n\n### Completed Tasks\n"
    output += "\n".join(complete) if complete else "No completed tasks.\n"
    return output

def generate_project_notes_section(project_name):
    """
    Generate a standardized "## Notes" section for a project page.
    Inserts a dynamic Dataview query that lists notes with the project's backlink.
    """
    query = (
        "> [!NOTE] **Notes**\n"
        "> ```dataview\n"
        "> LIST file.link\n"
        "> FROM \"PARA\"\n"
        "> WHERE type = \"note\" AND parent_area = this.file.name AND !contains(tags, \"#archive\")\n"
        "> SORT file.mtime DESC\n"
        "> ```\n"
    )
    output = "## Notes\n\n" + query
    return output

def generate_area_moc(area_name=None):
    """
    Generate new content for an area page as a Map of Content (MOC) using dynamic Dataview queries.
    The queries use "this.file.name" to list child areas, projects, and notes.
    The output is wrapped in callouts.
    The area_name parameter is accepted for test compatibility but is ignored.
    """
    moc = (
        "> [!NOTE] **Areas**\n"
        "> ```dataview\n"
        "> LIST file.link\n"
        "> FROM \"PARA\"\n"
        "> WHERE type = \"area\" AND parent_area = this.file.name AND !contains(tags, \"#archive\")\n"
        "> SORT file.mtime DESC\n"
        "> ```\n\n"
        "> [!NOTE] **Projects**\n"
        "> ```dataview\n"
        "> LIST file.link\n"
        "> FROM \"PARA\"\n"
        "> WHERE type = \"project\" AND parent_area = this.file.name AND !contains(tags, \"#archive\")\n"
        "> SORT file.mtime DESC\n"
        "> ```\n\n"
        "> [!NOTE] **Notes**\n"
        "> ```dataview\n"
        "> LIST file.link\n"
        "> FROM \"PARA\"\n"
        "> WHERE type = \"note\" AND parent_area = this.file.name AND !contains(tags, \"#archive\")\n"
        "> SORT file.mtime DESC\n"
        "> ```\n"
    )
    return moc

# --- YAML Frontmatter Update Function ---

def update_yaml_frontmatter(file_path, new_type, relationships, base_name):
    """
    Update the YAML frontmatter of the markdown file at file_path.
    new_type is one of: 'area', 'note', 'project', or 'task'.
    relationships is the merged CSV relationship dictionary for the page.
    base_name is the clean name of the page.
    
    For notes and projects, set:
      - parent_area: a backlink from the CSV "Area" or "Area/Resource" field (if available).
      - For notes, if available, a 'project' property is set.
    Also:
      - Remove any Archive, Area/Resource, or Area properties.
      - Archive is now represented solely via the "#archive" tag.
      - For projects, add a hierarchical tag: "#area/{slugify(parent_area)}" and "#project".
      - For notes, add tags "#area/{slugify(parent_area)}" and if in a project, "#project/{slugify(project_name)}".
      - For tasks, if associated with a project, add "#project"; always add "#task".
    """
    post = frontmatter.load(file_path)
    metadata = post.metadata

    # Force the type.
    metadata['type'] = new_type
    
    # Determine parent area from relationships.
    area_val = relationships.get("Area") or relationships.get("Area/Resource")
    if new_type in ['note', 'project', 'area'] and area_val:
        if isinstance(area_val, list):
            area_val = area_val[0]
        clean_area = extract_clean_name(area_val)
        # Ignore if the cleaned area is a URL.
        if not clean_area.lower().startswith("http"):
            parent_link = f"[[{clean_area}]]"
            metadata["parent_area"] = parent_link
            # Add hierarchical tag based on parent area.
            area_tag = f"#area/{slugify(clean_area)}"
        else:
            area_tag = "#area"
    else:
        area_tag = "#area"

    # For notes in projects, set project property.
    if new_type == 'note':
        proj_val = relationships.get("Project")
        if proj_val:
            if isinstance(proj_val, list):
                proj_val = proj_val[0]
            proj_clean = extract_clean_name(proj_val)
            metadata["project"] = f"[[{proj_clean}]]"
            proj_tag = f"#project/{slugify(proj_clean)}"
        else:
            proj_tag = ""
    elif new_type == 'project':
        # For projects, set project tag.
        proj_tag = f"#project"
    else:
        proj_tag = ""

    # Build final hierarchical tags (do not include note name in tag).
    final_tags = metadata.get("tags")
    if not final_tags:
        final_tags = []
    elif not isinstance(final_tags, list):
        final_tags = [final_tags]
    # Filter out None values.
    final_tags = [t for t in final_tags if t is not None]

    # For area, note, and project pages, add area tag (using parent area if available)
    if new_type in ['note', 'area', 'project']:
        if new_type == 'project' and "parent_area" in metadata:
            # For projects, hierarchical tag from parent area and project name.
            hierarchical_tag = f"#area/{slugify(extract_clean_name(relationships.get('Area') or ''))}"
        elif new_type in ['note', 'area'] and area_val:
            hierarchical_tag = f"{area_tag}"
        else:
            hierarchical_tag = f"{area_tag}"
        if hierarchical_tag not in final_tags:
            final_tags.append(hierarchical_tag)
            
    # For projects, add project tag.
    if new_type == 'project' and proj_tag and proj_tag not in final_tags:
        final_tags.append(proj_tag)
    # For notes, if in project, add project tag.
    if new_type == 'note' and proj_tag and proj_tag not in final_tags:
        final_tags.append(proj_tag)

    # Handle archive tag.
    if metadata['Archive'] and metadata['Archive'] in ['yes', 'true', 'x'] and "#archive" not in final_tags:
        final_tags.append("#archive")
    
    # Remove unwanted properties.
    for unwanted in ['Archive', 'Area/Resource', 'Area', 'Project']:
        metadata.pop(unwanted, None)

    # Process Archive: if relationships indicate archived, add "#archive" to tags.
    archive_val = str(relationships.get("Archive", "")).lower()
    archive_flag = archive_val in ['yes', 'true', 'x']
    
    # Handle archive tag.
    if archive_flag and "#archive" not in final_tags:
        final_tags.append("#archive")

    # Ensure all tags start with "#"
    final_tags = [t if t.startswith("#") else f"#{t}" for t in final_tags]
    metadata['tags'] = final_tags

    # Remove any 'created' property.
    metadata.pop('created', None)

    post.metadata = metadata
    return frontmatter.dumps(post)

# --- Unassigned Tasks Page Function ---

def create_unassigned_tasks_page(dest_folder, tasks_list):
    """
    Create an "Unassigned Tasks.md" file in the PARA folder for tasks that have no associated project.
    Always includes headers for "## General Tasks", "## Recurring Tasks", and "## Completed Tasks".
    """
    para_dir = os.path.join(dest_folder, "PARA")
    os.makedirs(para_dir, exist_ok=True)
    general = []
    recurring = []
    complete = []
    for task in tasks_list:
        line = "- "
        line += "[x] " if task['status'] else "[ ] "
        line += f"#task {task['task']}"
        if task.get('due'):
            line += f" (📅 {task['due']})"
        if task.get('recurring'):
            line += " (🔁 every week)"
            recurring.append(line)
        else:
            if task['status']:
                complete.append(line)
            else:
                general.append(line)
    content = ("---\n"
               "type: task\n"
               "tags:\n"
               "  - #task\n"
               "parent_area: []\n"
               "---\n\n"
               "# Unassigned Tasks\n\n")
    content += "## General Tasks\n"
    content += "\n".join(general) if general else "No tasks assigned.\n"
    content += "\n\n## Recurring Tasks\n"
    content += "\n".join(recurring) if recurring else "No recurring tasks assigned.\n"
    content += "\n\n## Completed Tasks\n"
    content += "\n".join(complete) if complete else "No completed tasks.\n"
    unassigned_path = os.path.join(para_dir, "Unassigned Tasks.md")
    with open(unassigned_path, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"Created Unassigned Tasks page with {len(general)+len(recurring)+len(complete)} tasks.")
    return unassigned_path

# --- Process Pages Function ---

def process_pages(source_folder, dest_folder, rels):
    """
    Process markdown pages from 'Areas-Resources', 'Notes', and 'Projects'.
    Update YAML frontmatter and replace page body content with standardized sections.
    New files are written into a flat "PARA" folder in dest_folder without modifying originals.
    """
    para_path = os.path.join(dest_folder, "PARA")
    os.makedirs(para_path, exist_ok=True)

    # Map pages from each category.
    categories = ['areas', 'notes', 'projects']
    mappings = { cat: find_markdown_pages(source_folder, cat) for cat in categories }

    area_map = map_relationships(rels.get('areas', []), 'Area/Resource')
    project_map = map_relationships(rels.get('projects', []), 'Name')
    note_map = map_relationships(rels.get('notes', []), 'Notes')

    # Process area and note pages.
    for cat in ['areas', 'notes']:
        for clean_name, file_path in mappings[cat].items():
            page_type = 'area' if cat == 'areas' else 'note'
            rel_info = area_map.get(clean_name, {}) if cat == 'areas' else note_map.get(clean_name, {})
            updated_content = update_yaml_frontmatter(file_path, page_type, rel_info, clean_name)
            if page_type == 'area':
                # Replace body with MOC content.
                header = updated_content.split('---', 2)[0] + '---\n'
                moc_content = generate_area_moc()
                updated_content = header + moc_content
            dest_file = os.path.join(para_path, os.path.basename(file_path))
            with open(dest_file, 'w', encoding='utf-8') as f:
                f.write(updated_content)
            # print(f"Processed '{os.path.basename(file_path)}' as {page_type} and wrote new file to PARA.")

    # Process project pages.
    for clean_name, file_path in mappings['projects'].items():
        page_type = 'project'
        rel_info = project_map.get(clean_name, {})
        updated_yaml = update_yaml_frontmatter(file_path, page_type, rel_info, clean_name)
        # Build new content: YAML header + inline tasks section + inline notes section.
        parts = updated_yaml.split('---', 2)
        yaml_header = parts[0] + '---\n'
        tasks_section = generate_project_tasks_section([
            # Gather tasks whose cleaned "Project" field equals the current project's clean name.
            {
                'task': extract_clean_name(task.get("Task", "")),
                'status': True if str(task.get("Done", "")).lower() in ['done', 'true', 'yes'] else False,
                'due': task.get("Due Date", None),
                'recurring': True if str(task.get("Recurring", "")).lower() in ['yes', 'true'] else False
            }
            for task in map_relationships(rels.get('tasks', []), 'Task').values()
            if extract_clean_name(task.get("Project", "")).lower() == clean_name
        ])
        notes_section = generate_project_notes_section(clean_name)
        new_body = "\n" + tasks_section + "\n\n" + notes_section
        new_content = yaml_header + new_body
        dest_file = os.path.join(para_path, os.path.basename(file_path))
        with open(dest_file, 'w', encoding='utf-8') as f:
            f.write(new_content)
        # print(f"Processed '{os.path.basename(file_path)}' as project and wrote new file to PARA.")

    # Process unassigned tasks (those with no associated project).
    task_map = map_relationships(rels.get('tasks', []), 'Task')
    unassigned_tasks = []
    for task_key, task_rel in task_map.items():
        if not task_rel.get('Project'):
            unassigned_tasks.append({
                'task': task_key,
                'status': True if str(task_rel.get("Done", "")).lower() in ['done', 'true', 'yes'] else False,
                'due': task_rel.get("Due Date", None),
                'recurring': True if str(task_rel.get("Recurring", "")).lower() in ['yes', 'true'] else False
            })
    create_unassigned_tasks_page(dest_folder, unassigned_tasks)

def load_relationships(source_folder):
    """
    Load CSV relationship files (Areas Resources.csv, Notes.csv, Projects.csv, Tasks.csv)
    from the source folder and return a dictionary with keys: 'areas', 'notes', 'projects', 'tasks'.
    """
    rels = {}
    file_map = {
        'areas': 'Areas Resources.csv',
        'notes': 'Notes.csv',
        'projects': 'Projects.csv',
        'tasks': 'Tasks.csv'
    }
    for key, filename in file_map.items():
        path = os.path.join(source_folder, filename)
        if os.path.exists(path):
            rels[key] = parse_csv_file(path)
        else:
            rels[key] = []
            print(f"Warning: {filename} not found in {source_folder}")
    return rels

def main():
    parser = argparse.ArgumentParser(description="Create Obsidian PARA structure with tag-based organization from Notion exports.")
    parser.add_argument("--source_folder", required=True, help="Path to the 'Database Storage' folder containing CSVs and markdown pages.")
    parser.add_argument("--dest_folder", required=True, help="Destination folder where the 'PARA' folder will be created.")
    args = parser.parse_args()

    rels = load_relationships(args.source_folder)
    print("Relationships loaded:")
    print(rels)
    process_pages(args.source_folder, args.dest_folder, rels)
    print("All pages processed and new files written into the PARA folder.")




In [66]:
import sys
sys.argv = [
    "migrate_para.py",  # dummy script name
    "--source_folder", r"C:\Users\leoro\OneDrive\Documents\Second Brain\database storage",
    "--dest_folder", r"C:\Users\leoro\OneDrive\Documents\Second Brain"
]


In [58]:
import unittest
import tempfile
import os
import shutil
import frontmatter

# Import the functions from your migration script.
# For example, if your script is named migrate_para.py, you can do:
# from migrate_para import (
#     slugify,
#     extract_clean_name,
#     parse_itemlist,
#     merge_relationship_rows,
#     update_yaml_frontmatter,
#     generate_project_tasks_section,
#     generate_project_notes_section,
#     generate_area_moc,
#     create_unassigned_tasks_page,
#     find_markdown_pages,
#     map_relationships,
#     process_pages,
#     load_relationships,
# )
#
# For this test suite, we assume they are already in the namespace.
# (Adjust the import statement as needed.)

# For integration tests using a real folder, set REAL_SOURCE_FOLDER accordingly.
REAL_SOURCE_FOLDER = r"C:\Users\leoro\OneDrive\Documents\Second Brain\database storage"

class TestMigrateParaUnit(unittest.TestCase):
    def test_extract_clean_name_notion(self):
        raw = "[[Database Storage/Areas-Resources/Muon Collider|Muon Collider]]"
        self.assertEqual(extract_clean_name(raw), "Muon Collider")

    def test_extract_clean_name_parenthesis(self):
        raw = "Modded Minecraft server (Notes%20139695d317a6400fb888739476fa8e51/Modded Minecraft server.html)"
        self.assertEqual(extract_clean_name(raw), "Modded Minecraft server")

    def test_parse_itemlist_with_comma_in_name(self):
        cell = "Distant Horizons mod for minecraft, install with tectonic (Notes%20139695d317a6400fb888739476fa8e51/Distant Horizons mod for minecraft, install with tectonic.html)"
        result = parse_itemlist(cell, marker="Notes%")
        self.assertEqual(result, ["Distant Horizons mod for minecraft, install with tectonic"])

    def test_merge_relationship_rows(self):
        rows = [
            {"Notes": "Note1"},
            {"Notes": "Note2"}
        ]
        merged = merge_relationship_rows(rows)
        self.assertEqual(merged.get("Notes"), ["Note1", "Note2"])

    def test_update_yaml_frontmatter_for_project(self):
        # Create a temporary markdown file with minimal YAML.
        sample_content = "---\nType: \nTags: []\n---\nOld content"
        with tempfile.NamedTemporaryFile("w+", suffix=".md", delete=False, encoding="utf-8") as tmp:
            tmp.write(sample_content)
            tmp_filename = tmp.name

        # Create a fake relationships dict for a project.
        # Here, we include an Area field (which should be used for a hierarchical tag)
        rel = {
            "Area": "Career",
            "Archive": "No",
            "Project": "Job Search"
        }
        updated = update_yaml_frontmatter(tmp_filename, "project", rel, "Job Search")
        post = frontmatter.loads(updated)
        # Check type forced to project.
        self.assertEqual(post.metadata.get("type"), "project")
        # Check that 'parent_area' is set correctly as a backlink.
        self.assertEqual(post.metadata.get("parent_area"), "[[Career]]")
        # Ensure that no redundant "Area" property remains.
        self.assertNotIn("Area", post.metadata)
        # Check hierarchical tags: should contain "#area/career/job_search" and "#project/job_search"
        tags = post.metadata.get("tags", [])
        self.assertTrue(any(tag == "#area/career/job_search" for tag in tags),
                        "Hierarchical area tag missing for project.")
        self.assertTrue(any(tag == "#project/job_search" for tag in tags),
                        "Project tag missing.")
        os.remove(tmp_filename)

    def test_update_yaml_frontmatter_for_note(self):
        sample_content = "---\nType: \nTags: []\n---\nOld note content"
        with tempfile.NamedTemporaryFile("w+", suffix=".md", delete=False, encoding="utf-8") as tmp:
            tmp.write(sample_content)
            tmp_filename = tmp.name

        # Fake relationship dict for a note with both Area and Project.
        rel = {
            "Area": "Health and Fitness",
            "Project": "Gym",
            "Archive": "No"
        }
        updated = update_yaml_frontmatter(tmp_filename, "note", rel, "Workout Notes")
        post = frontmatter.loads(updated)
        # Check type
        self.assertEqual(post.metadata.get("type"), "note")
        # Check that parent_area is set as backlink from Area
        self.assertEqual(post.metadata.get("parent_area"), "[[Health and Fitness]]")
        # Check that a project property is set as backlink.
        self.assertEqual(post.metadata.get("project"), "[[Gym]]")
        # Check tags include hierarchical tag for area and project tag.
        tags = post.metadata.get("tags", [])
        self.assertTrue(any(tag == "#area/health_and_fitness/workout_notes" for tag in tags),
                        "Hierarchical area tag missing for note.")
        self.assertTrue(any(tag == "#project/gym" for tag in tags),
                        "Project tag missing for note.")
        os.remove(tmp_filename)

    def test_update_yaml_frontmatter_for_task(self):
        sample_content = "---\nType: \nTags: []\nRecurring: Yes\nDone: Done\nTime Remaining (Days): -50 days\n---\nTask content"
        with tempfile.NamedTemporaryFile("w+", suffix=".md", delete=False, encoding="utf-8") as tmp:
            tmp.write(sample_content)
            tmp_filename = tmp.name

        rel = {
            "Project": "Job Search",
            "Archive": "No"
        }
        updated = update_yaml_frontmatter(tmp_filename, "task", rel, "Email A3D3")
        post = frontmatter.loads(updated)
        self.assertEqual(post.metadata.get("type"), "task")
        # Check that the Done property is now a boolean.
        self.assertTrue(isinstance(post.metadata.get("Done"), bool))
        # Check that Time Remaining (Days) is removed.
        self.assertNotIn("Time Remaining (Days)", post.metadata)
        # Check that recurring task tag is added.
        tags = post.metadata.get("tags", [])
        self.assertTrue(any(tag == "#tasks/recurring" for tag in tags),
                        "Recurring tag missing for task.")
        # Check that hierarchical task tag exists
        expected_tag = f"#project/{slugify('job search')}/email_a3d3"
        self.assertTrue(any(tag == expected_tag for tag in tags),
                        f"Expected hierarchical task tag '{expected_tag}' missing.")
        os.remove(tmp_filename)

    def test_generate_project_tasks_section(self):
        tasks = [
            {'task': "Call Physio & book appt", 'status': False, 'due': "2024-05-16", 'recurring': False},
            {'task': "Email A3D3", 'status': True, 'due': None, 'recurring': True}
        ]
        section = generate_project_tasks_section(tasks)
        self.assertIn("## Tasks", section)
        self.assertIn("### Incomplete Tasks", section)
        self.assertIn("### Completed Tasks", section)
        self.assertIn("[ ] #task Call Physio & book appt (📅 2024-05-16)", section)
        self.assertIn("[x] #task Email A3D3", section)
        self.assertIn("#tasks/recurring", section)

    def test_generate_project_notes_section(self):
        project_name = "Job Search"
        section = generate_project_notes_section(project_name)
        self.assertIn("## Notes", section)
        self.assertIn("FROM \"PARA\"", section)
        self.assertIn('parent_area = this.file.name', section)  # dynamic dataview query

    def test_generate_area_moc(self):
        moc = generate_area_moc()
        self.assertIn("LIST file.link", moc)
        self.assertIn("FROM \"PARA\"", moc)
        self.assertIn("this.file.name", moc)  # dynamic reference
        self.assertIn("**Areas**", moc)
        self.assertIn("**Projects**", moc)
        self.assertIn("**Notes**", moc)

    def test_create_unassigned_tasks_page(self):
        with tempfile.TemporaryDirectory() as tmp_dest:
            tasks = [
                {'task': "Task One", 'status': False, 'due': "2024-06-01", 'recurring': False},
                {'task': "Task Two", 'status': True, 'due': "2024-05-15", 'recurring': True}
            ]
            unassigned_path = create_unassigned_tasks_page(tmp_dest, tasks)
            self.assertTrue(os.path.exists(unassigned_path))
            with open(unassigned_path, "r", encoding="utf-8") as f:
                content = f.read()
            self.assertIn("## General Tasks", content)
            self.assertIn("## Recurring Tasks", content)
            self.assertIn("## Completed Tasks", content)

class TestMigrateParaIntegration(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        # Skip these tests if REAL_SOURCE_FOLDER does not exist.
        if not os.path.isdir(REAL_SOURCE_FOLDER):
            raise unittest.SkipTest(f"Real source folder '{REAL_SOURCE_FOLDER}' does not exist.")

    def setUp(self):
        # Create a temporary destination folder.
        self.tmp_dest = tempfile.TemporaryDirectory()
        self.para_path = os.path.join(self.tmp_dest.name, "PARA")
        rels = load_relationships(REAL_SOURCE_FOLDER)
        process_pages(REAL_SOURCE_FOLDER, self.tmp_dest.name, rels)

    def tearDown(self):
        self.tmp_dest.cleanup()

    def test_all_output_files_tags_format(self):
        # Verify that in every output file, all tags start with "#"
        for fname in os.listdir(self.para_path):
            fpath = os.path.join(self.para_path, fname)
            with open(fpath, "r", encoding="utf-8") as f:
                content = f.read()
            post = frontmatter.loads(content)
            tags = post.metadata.get("tags", [])
            for tag in tags:
                self.assertIsNotNone(tag)
                self.assertTrue(tag.startswith("#"), f"Tag '{tag}' in {fname} does not start with '#'.")

    def test_project_inline_tasks(self):
        # Check that every project file includes "## Tasks" and "## Notes" sections.
        project_found = False
        for fname in os.listdir(self.para_path):
            fpath = os.path.join(self.para_path, fname)
            post = frontmatter.load(fpath)
            if post.metadata.get("type", "").lower() == "project":
                project_found = True
                with open(fpath, "r", encoding="utf-8") as f:
                    content = f.read()
                self.assertIn("## Tasks", content, f"Project file {fname} missing '## Tasks' section.")
                self.assertIn("## Notes", content, f"Project file {fname} missing '## Notes' section.")
        self.assertTrue(project_found, "No project files found in output.")

    def test_unassigned_tasks_page_created(self):
        unassigned_path = os.path.join(self.para_path, "Unassigned Tasks.md")
        self.assertTrue(os.path.exists(unassigned_path), "Unassigned Tasks.md was not created.")
        with open(unassigned_path, "r", encoding="utf-8") as f:
            content = f.read()
        self.assertIn("## General Tasks", content, "General Tasks section missing in Unassigned Tasks page.")
        self.assertIn("## Recurring Tasks", content, "Recurring Tasks section missing in Unassigned Tasks page.")
        self.assertIn("## Completed Tasks", content, "Completed Tasks section missing in Unassigned Tasks page.")

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


..FF...F..

Created Unassigned Tasks page with 3 tasks.
Appended 2 tasks to tmp3xj7sdib.md.


F

Created Unassigned Tasks page with 0 tasks.


F

...

Created Unassigned Tasks page with 0 tasks.


s

Created Unassigned Tasks page with 0 tasks.


.
ERROR: test_generate_area_moc (__main__.TestMigrateParaMOC)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\leoro\AppData\Local\Temp\ipykernel_52384\1109567064.py", line 18, in test_generate_area_moc
    moc = generate_area_moc(area_name)
TypeError: generate_area_moc() takes 0 positional arguments but 1 was given

ERROR: test_update_yaml_frontmatter_area_page_replacement (__main__.TestMigrateParaMOC)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\leoro\AppData\Local\Temp\ipykernel_52384\1109567064.py", line 38, in test_update_yaml_frontmatter_area_page_replacement
    moc_content = generate_area_moc("Self Advice")
TypeError: generate_area_moc() takes 0 positional arguments but 1 was given

ERROR: test_all_output_files_tags_format (__main__.TestRealOutputFiles)
----------------------------------------------------------------------
Trace

Created Unassigned Tasks page with 0 tasks.
Unassigned Tasks page content (first 500 chars):
 ---
type: task
tags:
  - #task
parent_area: []
---

# Unassigned Tasks

## General Tasks
No tasks assigned.

## Recurring Tasks
No recurring tasks assigned.

## Completed Tasks
No completed tasks.



In [67]:
if __name__ == "__main__":
    main()


Relationships loaded:
{'areas': [{'Area/Resource': 'Hobbies', 'Type': 'Area', 'Resources': 'Music', 'Root Area': 'Personal Development/Growth', 'Archive': 'No'}, {'Area/Resource': 'Movies', 'Type': 'Resource', 'Notes': 'Streaming Sites', 'Root Area': 'Hobbies', 'Archive': 'No'}, {'Area/Resource': 'Personal Development/Growth', 'Type': 'Area', 'Resources': 'Hobbies', 'Notes': 'Lust vs Love', 'Projects': 'Driving!', 'Archive': 'No'}, {'Area/Resource': 'Music', 'Type': 'Resource', 'Notes': 'Jazz Notes', 'Root Area': 'Hobbies', 'Archive': 'No'}, {'Area/Resource': 'Books', 'Type': 'Resource', 'Notes': 'Good Authors!', 'Root Area': 'Hobbies', 'Archive': 'No'}, {'Area/Resource': 'Health and Fitness', 'Type': 'Area', 'Resources': 'Gym', 'Root Area': 'Personal Development/Growth', 'Archive': 'No'}, {'Area/Resource': 'Gym', 'Type': 'Resource', 'Notes': 'Old notes', 'Root Area': 'Health and Fitness', 'Archive': 'No'}, {'Area/Resource': 'Mental Health', 'Type': 'Area', 'Resources': 'Therapy', 'Roo